In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import warnings
import time

warnings.filterwarnings('ignore')

# ==============================================================================
# KONFIGURASI AWAL
# ==============================================================================
FILE_BBM = "BBM AAB.xlsx"
FILE_DOORING = "DOORING OKT-DES 2025 (Copy).xlsx"
FILE_HAULAGE = "HAULAGE OKT-DES 2025 (Copy).xlsx"
FILE_TLP = "dashboard TLP okt-des (S1L Trucking).xlsx"

VALHALLA_URL = "http://localhost:8002/route"
DEPO_LAT = -7.2269  # Koordinat Depo PT SPIL YON
DEPO_LON = 112.7307

HAULAGE_DISTANCE_MAP = {
    "DEPO - DEPO": 5,
    "BERLIAN - DEPO": 10,
    "TTL - DEPO": 15,
    "DEPO - TELUK LAMONG": 20,
    # Tambahkan rute manual Haulage lainnya di sini jika ada
}

# ==============================================================================
# FUNGSI PERHITUNGAN BERAT
# ==============================================================================
def hitung_berat_dooring(size_cont_str):
    size_str = str(size_cont_str).upper()
    W_20_EMPTY, W_20_FULL = 2300, 27000
    W_40_EMPTY, W_40_FULL = 3800, 32000
    total_berat = 0
    
    if 'COMBO' in size_str:
        total_berat = (2 * W_20_EMPTY) + (2 * W_20_FULL)
    elif '40' in size_str: 
        total_berat = W_40_EMPTY + W_40_FULL
    else: 
        multiplier = 2 if '2X' in size_str else 1
        total_berat = (multiplier * W_20_EMPTY) + (multiplier * W_20_FULL)
        
    return total_berat / 1000 # Konversi ke Ton

def hitung_berat_haulage(row):
    kegiatan = str(row['Jenis Kegiatan']).upper()
    jenis_cont = str(row['JENIS CONT']).upper()
    status = str(row['FULL/EMPTY']).upper()
    
    if any(x in kegiatan for x in ['MOVE ARMADA', 'KIR']): return 0
    if 'LOWBED' in jenis_cont or 'DOLLY' in jenis_cont: return 0
    
    w_empty, w_full = 0, 0
    if '20FT' in jenis_cont:
        w_empty, w_full = 2300, 27000
    elif '40FT' in jenis_cont:
        w_empty, w_full = 3800, 32000
    elif 'GENSET' in jenis_cont:
        w_full, w_empty = 2300, 0
    
    final_weight = w_full if status == 'FULL' else (w_empty if status == 'EMPTY' else 0)
    return final_weight / 1000 # Konversi ke Ton

# ==============================================================================
# PROSES 1: MAPPING UNIT & LOAD BBM (OKT-NOV)
# ==============================================================================
print("1. Memproses Data BBM & Mapping Unit...")
df_bbm = pd.read_excel(FILE_BBM, sheet_name=None)
df_bbm_comb = pd.concat([
    df_bbm['OKT'].assign(Bulan='Oktober'),
    df_bbm['NOV'].assign(Bulan='November')
], ignore_index=True)

kpi_ref = df_bbm_comb[['GROUP KPI', 'EQUIP NAME']].dropna().drop_duplicates()
kpi_ref['CLEAN_KPI'] = kpi_ref['GROUP KPI'].astype(str).str.strip().str.upper()
kpi_map = dict(zip(kpi_ref['CLEAN_KPI'], kpi_ref['EQUIP NAME']))

bbm_per_unit = df_bbm_comb.groupby('EQUIP NAME')['LITER'].sum().reset_index()

1. Memproses Data BBM & Mapping Unit...


KeyboardInterrupt: 

In [ ]:
# ==============================================================================
# PROSES 2: PROSES DATA DOORING (BERAT, JARAK VALHALLA, TON-KM)
# ==============================================================================
print("2. Memproses Data Dooring...")
df_door_okt = pd.read_excel(FILE_DOORING, sheet_name='OKT')
df_door_nov = pd.read_excel(FILE_DOORING, sheet_name='NOV')
df_dooring = pd.concat([df_door_okt, df_door_nov], ignore_index=True)

df_dooring['Clean_Lambung'] = df_dooring['LAMBUNG'].astype(str).str.strip().str.upper()
df_dooring['Unit_Name'] = df_dooring['Clean_Lambung'].map(kpi_map)
df_dooring['Berat_Ton'] = df_dooring['SIZE CONT'].apply(hitung_berat_dooring)

df_tlp_okt = pd.read_excel(FILE_TLP, sheet_name='oktober')[['SOPT_NO', 'DOORING_ADDRESS']].rename(columns={'SOPT_NO': 'NO_SOPT', 'DOORING_ADDRESS': 'ALAMAT'})
df_tlp_nov = pd.read_excel(FILE_TLP, sheet_name='november')[['SOPT NO', 'PICKUP / DELIVERY ADDRESS']].rename(columns={'SOPT NO': 'NO_SOPT', 'PICKUP / DELIVERY ADDRESS': 'ALAMAT'})
df_ref_tlp = pd.concat([df_tlp_okt, df_tlp_nov]).drop_duplicates(subset=['NO_SOPT'])

df_dooring = pd.merge(df_dooring, df_ref_tlp, left_on='NO. SOPT 1', right_on='NO_SOPT', how='left')

# --- Geocoding & Valhalla Local Routing ---
print("   -> Memulai penarikan jarak via Valhalla Server Lokal...")
geolocator = Nominatim(user_agent="trucking_calc_spil")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

list_jarak = []
cache_koordinat = {}

for idx, row in df_dooring.iterrows():
    alamat = row['ALAMAT']
    jarak_km = 0
    
    if pd.notna(alamat) and alamat != "-":
        if alamat in cache_koordinat:
            lat, lon = cache_koordinat[alamat]
        else:
            try:
                loc = geocode(str(alamat))
                lat, lon = (loc.latitude, loc.longitude) if loc else (None, None)
                cache_koordinat[alamat] = (lat, lon)
            except:
                lat, lon = None, None
        
        if lat and lon:
            json_body = {
                "locations": [
                    {"lat": DEPO_LAT, "lon": DEPO_LON},
                    {"lat": lat, "lon": lon},
                    {"lat": DEPO_LAT, "lon": DEPO_LON}
                ],
                "costing": "truck",
                "units": "km"
            }
            try:
                resp = requests.post(VALHALLA_URL, json=json_body, timeout=5)
                if resp.status_code == 200:
                    jarak_km = resp.json()['trip']['summary']['length']
            except:
                jarak_km = 0
                
    list_jarak.append(jarak_km)
    if idx % 50 == 0 and idx > 0:
        print(f"   -> Selesai memproses {idx} rute dooring...")

df_dooring['Jarak_Km'] = list_jarak
df_dooring['TonKm_Dooring'] = df_dooring['Berat_Ton'] * df_dooring['Jarak_Km']

grp_dooring = df_dooring.groupby('Unit_Name').agg({
    'Berat_Ton': 'sum',
    'Jarak_Km': 'sum',
    'TonKm_Dooring': 'sum'
}).reset_index()
grp_dooring.rename(columns={'Berat_Ton': 'Ton_Dooring', 'Jarak_Km': 'Km_Dooring'}, inplace=True)



In [ ]:
# ==============================================================================
# PROSES 3: PROSES DATA HAULAGE (BERAT, JARAK MANUAL, TON-KM)
# ==============================================================================
print("3. Memproses Data Haulage...")
df_haulage = pd.read_excel(FILE_HAULAGE)
df_haulage['TANGGAL'] = pd.to_datetime(df_haulage['TANGGAL'])
df_haulage = df_haulage[df_haulage['TANGGAL'].dt.month.isin([10, 11])]

df_haulage['Clean_Lambung'] = df_haulage['NO LAMBUNG'].astype(str).str.strip().str.upper()
df_haulage['Unit_Name'] = df_haulage['Clean_Lambung'].map(kpi_map)

df_haulage['Berat_Ton'] = df_haulage.apply(hitung_berat_haulage, axis=1)
df_haulage['Jarak_Km'] = df_haulage['RUTE'].map(HAULAGE_DISTANCE_MAP).fillna(0)
df_haulage['TonKm_Haulage'] = df_haulage['Berat_Ton'] * df_haulage['Jarak_Km']

grp_haulage = df_haulage.groupby('Unit_Name').agg({
    'Berat_Ton': 'sum',
    'Jarak_Km': 'sum',
    'TonKm_Haulage': 'sum'
}).reset_index()
grp_haulage.rename(columns={'Berat_Ton': 'Ton_Haulage', 'Jarak_Km': 'Km_Haulage'}, inplace=True)


In [ ]:
# ==============================================================================
# PROSES 4: FINAL MERGE & ANALISA EFISIENSI
# ==============================================================================
print("4. Menggabungkan Semua Data (Finalizing)...")
df_ops = pd.merge(grp_dooring, grp_haulage, on='Unit_Name', how='outer').fillna(0)

df_ops['Total_Ton'] = df_ops['Ton_Dooring'] + df_ops['Ton_Haulage']
df_ops['Total_Km'] = df_ops['Km_Dooring'] + df_ops['Km_Haulage']
df_ops['Total_TonKm'] = df_ops['TonKm_Dooring'] + df_ops['TonKm_Haulage']

df_final = pd.merge(df_ops, bbm_per_unit, left_on='Unit_Name', right_on='EQUIP NAME', how='inner')

df_final['L_per_Km'] = np.where(df_final['Total_Km'] > 0, df_final['LITER'] / df_final['Total_Km'], 0)
df_final['L_per_Ton'] = np.where(df_final['Total_Ton'] > 0, df_final['LITER'] / df_final['Total_Ton'], 0)
df_final['L_per_TonKm'] = np.where(df_final['Total_TonKm'] > 0, df_final['LITER'] / df_final['Total_TonKm'], 0)


# ==============================================================================
# PROSES 5: EXPORT KE EXCEL
# ==============================================================================
file_output = "HASIL_ANALISA_TRUCKING_OKT_NOV.xlsx"
df_final.to_excel(file_output, index=False)
print(f"Selesai! File berhasil disimpan dengan nama: {file_output}")

# Menampilkan 5 baris pertama sebagai preview di Jupyter
display(df_final.head())